In [ ]:

# ============================================================
# 시스템 환경 설정 (Colab 및 로컬 분기 안전)
# ============================================================

import os, sys

def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_colab():
    print("Colab 환경 감지: 필요한 패키지를 설치합니다.")
    !pip install --quiet openai langchain chromadb pypdf tiktoken
    !pip show openai langchain chromadb | grep Version
else:
    print("로컬(Windows/Conda) 환경 감지: 기존 설치 환경을 유지합니다.")

from getpass import getpass
import openai

if "OPENAI_API_KEY" not in os.environ:
    openai.api_key = getpass("Enter your OpenAI API key: ")
else:
    openai.api_key = os.environ["OPENAI_API_KEY"]

print(f"현재 경로: {os.getcwd()}")
print("✅ 환경 설정 완료")


In [ ]:

# ============================================================
# 실습 1. Embedding 기본 (소방기본법)
# ============================================================

from google.colab import files
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

uploaded = files.upload()
loader = PyPDFLoader("소방기본법.pdf")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)

embedding = OpenAIEmbeddings(model="text-embedding-3-large")
db = Chroma.from_documents(chunks, embedding)

query = "소방기본법의 목적은 무엇인가?"
results = db.similarity_search(query, k=3)

for i, r in enumerate(results):
    print(f"[{i+1}] {r.page_content[:200]}")

# --- 옵션 실습 ---
# 1. query를 "소방기본법 제3조의 정의는?" 로 바꾸어 실행해보세요.
# 2. splitter의 chunk_size를 1500으로 바꾸고 결과 차이를 확인해보세요.


In [ ]:

# ============================================================
# 실습 2. RAG 확장 (소방기본법 + 화재예방조례)
# ============================================================

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

retriever = db.as_retriever(search_kwargs={"k":3})
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

query = "화재예방조례에 따른 대피시설 설치 기준은?"
print(qa.run(query))

# --- 옵션 실습 ---
# 1. query를 "소방기본법 제25조와 화재예방조례 제9조의 차이점은?" 로 변경해보세요.
# 2. retriever의 search_kwargs 값을 {'k': 5}로 바꾸어 더 많은 문서 검색을 수행해보세요.


In [ ]:

# ============================================================
# 실습 3. 자동 보고서 생성
# ============================================================

prompt_template = '''
아래는 소방 법령 질의응답 결과입니다.
이를 바탕으로 보고서 형식으로 요약해 주세요.

[질의]
{question}

[응답]
{answer}

형식:
- 관련 법령:
- 주요 내용 요약:
- 조치 및 권고사항:
'''

query = "화재예방조례의 주요 벌칙 조항을 보고서 형태로 정리해줘."
answer = qa.run(query)
report = prompt_template.format(question=query, answer=answer)
print("자동 보고서 생성 결과:")
print(llm.invoke(report).content)

# --- 옵션 실습 ---
# 1. 템플릿에 '참고 조문' 항목을 추가해보고 결과를 비교해보세요.
# 2. report 내용을 파일(txt)로 저장해보세요.
